### PathMovers

This notebook is an introduction to handling `PathMover` and `PathMoveChange` instances. It mostly covers the questions on 
1. how to check if a certain mover was part of a change. 
2. What are the possible changes a mover can generate.
3. ...

Load OPENPATHSAMPLING

In [1]:
import openpathsampling as p

Let's open a file that contains a mover and lot's of changes

In [2]:
st = p.storage.Storage('mstis.nc', mode='r')

A Simulator creates simulation steps. So we load a single step.

In [3]:
mc = st.steps[3]
print mc

Each step posesses a pathmovechange which we will get

In [4]:
pmc = mc.change
print pmc

PathSimulatorStep : PathSampling : Step # 3 with 0 samples
 +- RandomChoice :
     +- RandomChoice :
         +- SampleMove : ReplicaExchangeMover : False :[]


And the mover that was used to generate this change

In [5]:
pm = pmc.mover
print pm.treeprint()

PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ShootingChooser(10f3af810)
     |   +- OneWayShootingMover I'face 0(10f3c79d0)
     |   |   +- ForwardShoot(1060e4e50)
     |   |   +- BackwardShoot(10f3c7f10)
     |   +- OneWayShootingMover I'face 1(1060f5850)
     |   |   +- ForwardShoot(10f0b2110)
     |   |   +- BackwardShoot(10f3d0e10)
     |   +- OneWayShootingMover I'face 2(10f3c7150)
     |   |   +- ForwardShoot(10f3c0050)
     |   |   +- BackwardShoot(10f3c7ad0)
     |   +- OneWayShootingMover I'face 0(10f3af610)
     |   |   +- ForwardShoot(10f3c0990)
     |   |   +- BackwardShoot(10f3af3d0)
     |   +- OneWayShootingMover I'face 1(10f3c7390)
     |   |   +- ForwardShoot(10f3d0290)
     |   |   +- BackwardShoot(10f3c7050)
     |   +- OneWayShootingMover I'face 2(10f3aff50)
     |   |   +- ForwardShoot(10f3c0450)
     |   |   +- BackwardShoot(10f3af210)
     |   +- OneWayShootingMover I'face 0(10f3c7090)
     |   |   +- ForwardShoot(10f3d0d90)
     |   |   +- Backwa

Let's first check if the pmc was really created by the pathmover

In [6]:
pmc in pm

True

This should be obvious, but under the hood there is some more fancy stuff happening. The `in` keyword with pathmovers and changes actually works on trees of movers. These trees are used to represent a specific order or movers being called and these trees are unique for changes. This means there is a way to label each possible pmc that can be generated by a mover. This possible only refers to choices in actual movers. 

This is due to special PathMovers that can pick one or more movers among a set of possibilities. The simplest of these is the OneWayShooter which choses equal between forward and backward shooting. So a OneWayShooter has two possible outcomes. Let's look at these

In [7]:
ow_mover = p.OneWayShootingMover([], []) # we use dummy arguments since are not going to use it

The property `enum` will evaluate the (potentially VERY large) number of all possible changes

In [8]:
list(ow_mover.enum)

[(OneWayShooting(10f3c7b50), (ForwardShoot(10f3c7350))),
 (OneWayShooting(10f3c7b50), (BackwardShoot(10f3c7f50)))]

which is two in our case. Another example

In [9]:
ow_mover_2 = p.RandomChoiceMover([
        p.OneWayShootingMover([], []),
        p.OneWayShootingMover([], [])
        ])

In [10]:
list(ow_mover_2.enum)

[(RandomChoice(10f60d790),
     (OneWayShooting(10f3c0b10), (BackwardShoot(10f60d750)))),
 (RandomChoice(10f60d790),
     (OneWayShooting(10f3c0b10), (ForwardShoot(10f0b2590)))),
 (RandomChoice(10f60d790),
     (OneWayShooting(10f3c0610), (ForwardShoot(10f60d050)))),
 (RandomChoice(10f60d790),
     (OneWayShooting(10f3c0610), (BackwardShoot(10f60d410))))]

Now we have 4 choices as expected 2 for the RandomChoice * 2 for the OneWayShooter. Although effectively there are only 2 distinct ones. Why is that? The problem in `ow_mover_2` is that we defined two separate instances of the OneWayShootingMover and in general different instances are considered different. In this case it might even be possible to check for equality, but we decided to leave this to the user. If you create two instances we assume there is a reason. Maybe just calling the differently (although it makes no sense for the monte carlo scheme).

The next example uses the same mover twice

In [11]:
ow_mover_3 = p.RandomChoiceMover([
        ow_mover,
        ow_mover
        ])

In [12]:
list(ow_mover_3.enum)

[(RandomChoice(10f3c0350),
     (OneWayShooting(10f3c7b50), (ForwardShoot(10f3c7350)))),
 (RandomChoice(10f3c0350),
     (OneWayShooting(10f3c7b50), (BackwardShoot(10f3c7f50))))]

And we get only two distinct movers as we wanted to.

Let's look at the real mover

In [13]:
all_changes = list(pm.enum)
print len(all_changes)

63


This one has 63. Let's if we can reconstruct that number. The pathmover first choses between 5 different move types. 

1. Shooting, with one shooter for each of the 3 x 3 + 1? ensembles and two directions (forward and backward) are 10 * 2 = 20 in total. 
2. Reversal moves for each ensemble = 10
3. ReplicaExchange moves between all neighboring ensembles 3 x [1,2], [2,3]  = 6
4. OuterRexEx for each state = 3
5. And a choice of a Minus Move for each state that consists of choices for picking the first or final inner trajectory from the minus ensemble and 2 choices for extending forward or backward. Here is important that we use a ConditionalSequentialMover that might call only some of its inner sequence. To be precise it could call only the subtrajectoryselection (2 choices), the selection and the replicaexchange (also 2 choices) or all three movers (4 choices). This will give a total of 8 possibilities for the minus move per state and thus = 24.

In total we have 20 + 10 + 6 + 3 + 24 = 63 as `enum` predicted. 

> Note that this is only the maximal possible number of moves and not the real accessible moves. It could be that in a conditional mover some moves are always run in sequence. E.g. in our case the subtrajectoryselection should always work and hence the first 2 choices of the minus mover will never happen. It will always try a swap (which can fail if by chance the trajectory in the innermost interface crosses away from the state). So the effective number of choices will be 57 and not 63.

Let's do some more testing

First let's check if the keys we get for all changes in the storage are actually contained in the mover or put more simple the mover could have generated these changes. This should be true for all changes that originate from our mover.

In [14]:
print [pc in pm for pc in st.pathmovechanges[0:20]]

[False, False, True, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, True, False]


What happened here? Why are only some changes in the storage made by our mover? Shouldn't all of them (except the 2 in the beginning) be the result of our pathmover? Yes and no. They are, but we are checking if the total change is made from a mover and we are also loading all subchanges.

In [15]:
print st.pathmovechanges[2]
print st.pathmovechanges[2] in pm
print
print st.pathmovechanges[5]
print st.pathmovechanges[5] in pm

PathSimulatorStep : PathSampling : Step # 1 with 2 samples
 +- RandomChoice :
     +- RandomChoice :
         +- SampleMove : ReplicaExchangeMover : True : 2 samples [<Sample @ 0x10f0b2390>, <Sample @ 0x10f0b2090>]
True

SampleMove : ReplicaExchangeMover : True : 2 samples [<Sample @ 0x10f0b2390>, <Sample @ 0x10f0b2090>]
False


We will now cache all changes to see if the next steps are really fast or not.

In [16]:
_ = list(st.pathmovechanges)
_ = list(st.steps)

Now the question is: 
How do we get all the changes that are really important? One way is to scan all changes and use only the ones from the mover

In [17]:
real_changes = filter(lambda x : x in pm, st.pathmovechanges)
print len(real_changes), 'of', len(st.pathmovechanges)

300 of 1693


While this works it would be better to rely on the steps in the simulation and not on the changes itself. We might store some other changes for whatever reason, but we only want the ones that are associated with a MC step. The steps in the storage do exactly that and point to the changes that are used to generate the next sampleset that we wanted. 

In [18]:
step_changes = [step.change for step in st.steps[1:]]
print len(step_changes)

300


We exclude the first step since it is the initialization which is not generated by our pathmover.

Let's now check which and how often one the 31 possible steps was called.

In [19]:
import collections
counter = collections.defaultdict(lambda: 0)
for ch in step_changes:
    counter[ch.unique] += 1

In [20]:
s = '%d of %d different changes run' % (len(counter), len(list(pm.enum)))
print s, '\n', '-' * len(s), '\n'

for y in sorted(counter.items(), key=lambda x : -x[1]):
    print
    print y[1], 'x'
    print y[0].treeprint()

45 of 63 different changes run 
------------------------------ 


16 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- MSOuterRepexChooser(10f3bbb90)
         +- ReplicaExchange(10f3bb590)

14 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ReversalChooser(10f3af6d0)
         +- PathReversal(10f3c7690)

14 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- MSOuterRepexChooser(10f3bbb90)
         +- ReplicaExchange(10f60d3d0)

14 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- MSOuterRepexChooser(10f3bbb90)
         +- ReplicaExchange(10f3bb190)

12 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ReversalChooser(10f3af6d0)
         +- PathReversal(10f3c7550)

11 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ShootingChooser(10f3af810)
         +- OneWayShootingMover I'face 1(10f3af310)
             +- BackwardShoot(10f3afc10)

10 x
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ShootingCh

We see that especially minus moves are underrepresented. This is due to the standard weighting of the minus move which runs minus moves much less frequent than other moves. 

Let's just do that again and pick a random chance among all the possibilities. This is done only from the possibilities without regard for acceptance etc. We only assume that all possible choices for a single mover are equally likely.

In [21]:
pmc_list = [pm.random() for x in xrange(10000)]

In [22]:
counter2 = collections.defaultdict(lambda: 0)
for ch in pmc_list:
    counter2[ch] += 1

In [23]:
s = '%d of %d different changes run' % (len(counter2), len(list(pm.enum)))
print s, '\n', '-' * len(s), '\n'

for y in sorted(counter2.items(), key=lambda x : -x[1]):
    print (100.0 * y[1]) / len(pmc_list), '%', repr(y[0])

63 of 63 different changes run 
------------------------------ 

6.98 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (MSOuterRepexChooser(10f3bbb90), (ReplicaExchange(10f3bb190),))))
6.81 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (MSOuterRepexChooser(10f3bbb90), (ReplicaExchange(10f3bb590),))))
6.45 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (MSOuterRepexChooser(10f3bbb90), (ReplicaExchange(10f60d3d0),))))
3.6 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (RepExChooser(10f3afa50), (ReplicaExchange(10f3a76d0),))))
3.41 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (RepExChooser(10f3afa50), (ReplicaExchange(10f3af650),))))
3.37 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (RepExChooser(10f3afa50), (ReplicaExchange(10f3af390),))))
3.28 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (RepExChooser(10f3afa50), (ReplicaExchange(10f3afbd0),))))
3.25 % (PathSimulator(10f60df50), (RandomChoice(10f3bb990), (RepExChooser(10f3afa5

We realize that a specific shooter is called less likely than other movers

> not sure if this is helpful. We could have additional weighting factors for the randomchoice, but each mover would have to know about it. Problem is that in most movers these weighting probabilities are dependent on the context and hence make no sense here!

> One solution is to let a mover be able to give a formal expression for each possible choice, since each mover will only depend on 
1. results of accepted/rejected of submoves and ultimately if certain samples are accepted which in turn depends on `.valid` and `.proposal_probability`
2. some random numbers (choices)

> So the SampleGeneratingMovers will return something like `[acc(samp[]), acc(samp[])]`

> while other movers will return something that depends on the submovers, it could just be a list of products.

> Each mover has an expression for it being accepted

> Each mover has an expression for picking a specific choice

> Finally `.random()` will use the product of all probabilities and a specific value for accepting

PathMovers and PathMoveChanges now have a `.treeprint()` function

In [24]:
print pm.treeprint()

PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ShootingChooser(10f3af810)
     |   +- OneWayShootingMover I'face 0(10f3c79d0)
     |   |   +- ForwardShoot(1060e4e50)
     |   |   +- BackwardShoot(10f3c7f10)
     |   +- OneWayShootingMover I'face 1(1060f5850)
     |   |   +- ForwardShoot(10f0b2110)
     |   |   +- BackwardShoot(10f3d0e10)
     |   +- OneWayShootingMover I'face 2(10f3c7150)
     |   |   +- ForwardShoot(10f3c0050)
     |   |   +- BackwardShoot(10f3c7ad0)
     |   +- OneWayShootingMover I'face 0(10f3af610)
     |   |   +- ForwardShoot(10f3c0990)
     |   |   +- BackwardShoot(10f3af3d0)
     |   +- OneWayShootingMover I'face 1(10f3c7390)
     |   |   +- ForwardShoot(10f3d0290)
     |   |   +- BackwardShoot(10f3c7050)
     |   +- OneWayShootingMover I'face 2(10f3aff50)
     |   |   +- ForwardShoot(10f3c0450)
     |   |   +- BackwardShoot(10f3af210)
     |   +- OneWayShootingMover I'face 0(10f3c7090)
     |   |   +- ForwardShoot(10f3d0d90)
     |   |   +- Backwa

In [25]:
print [m.name for key, m in pm.iteritems()]

['PathSimulator', 'BackwardExtend', 'ForwardShoot', 'EnsembleFilter', 'EnsembleFilter', 'BackwardShoot', 'PathReversal', u'InterfaceSetChooser', 'Minus', 'PathReversal', 'ForwardShoot', 'ForwardExtend', 'FinalSubtrajectorySelect', u'MinusExtensionDirectionChooser', 'ReplicaExchange', 'PathReversal', 'ForwardShoot', u"OneWayShootingMover I'face 0", 'ForwardShoot', 'ReplicaExchange', 'ReplicaExchange', 'ConditionalSequential', 'BackwardShoot', 'ForwardShoot', 'ReplicaExchange', 'BackwardShoot', 'ReplicaExchange', 'ConditionalSequential', 'FirstSubtrajectorySelect', 'PathReversal', 'BackwardShoot', u"OneWayShootingMover I'face 0", 'ForwardShoot', u"OneWayShootingMover I'face 1", u'InterfaceSetChooser', u'MinusExtensionDirectionChooser', 'OneWayShooting', 'ForwardShoot', 'ConditionalSequential', 'ReplicaExchange', 'FirstSubtrajectorySelect', u'MinusSubtrajectoryChooser', 'PathReversal', u'MinusSubtrajectoryChooser', 'BackwardShoot', 'BackwardShoot', 'FirstSubtrajectorySelect', u"OneWayShoo

Get the (unique) location of the randomchoicemover. You can search for Mover classes, Mover instances or by the `.name` property of a mover which is a string.

In [26]:
loc_rc = pm.locate("OneWayShootingMover I'face 2")[0]
print loc_rc

PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- ShootingChooser(10f3af810)
         +- OneWayShootingMover I'face 2(10f3c7150)


In [27]:
loc_rc = pm.locate('RandomChoice')
print loc_rc

PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)


the location object is effectively a tree mover instances described by nested tuples. For convenience it is wrapped to make searching easier and format the output. 

In [28]:
print type(loc_rc)
print isinstance(loc_rc, tuple)
print repr(loc_rc)
print str(loc_rc)

<class 'openpathsampling.treelogic.TupleTree'>
True
(PathSimulator(10f60df50), (RandomChoice(10f3bb990),))
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)


In most cases you can use python tuples instead of TupleTree. The structure of a tree of tuples looks as follows:

Each node has the following structure

    (head, (child1, ), (child2, ), ...)
    
where head is a the node content (almost always a `Mover` instance) and child is again a node. It is important to note that a tuple tree does not support choices of different children like a real pathmover can. Therefore we could generate a tree tuple of the content of a pathmover but it will not reflect the possible choices in it. To get the associated tree tuple of a change (which has no choice in it) we can use `.unique`.

In [29]:
print pmc
print repr(pmc.unique)

PathSimulatorStep : PathSampling : Step # 3 with 0 samples
 +- RandomChoice :
     +- RandomChoice :
         +- SampleMove : ReplicaExchangeMover : False :[]
(PathSimulator(10f60df50), (RandomChoice(10f3bb990), (MSOuterRepexChooser(10f3bbb90), (ReplicaExchange(10f60d3d0),))))


Instead of checking for a pmc directly we can also check for the tuple tree representation

In [30]:
print pmc
print pmc in pm # check if pmc could have been generated by pm
print pmc.unique in pm # check if the tuple tree representation could have been generated by pm

PathSimulatorStep : PathSampling : Step # 3 with 0 samples
 +- RandomChoice :
     +- RandomChoice :
         +- SampleMove : ReplicaExchangeMover : False :[]
True
True


Now get the mover at the `loc_rc` location (which is of course a RandomChoiceMover).

In [31]:
rc = pm[loc_rc]

These are the weights by mover

In [32]:
dict(zip(rc.movers, rc.weights))

{MinusChooser(10f3a71d0): 0.6000000000000001,
 ReversalChooser(10f3af6d0): 5,
 ShootingChooser(10f3af810): 10,
 RepExChooser(10f3afa50): 3,
 MSOuterRepexChooser(10f3bbb90): 3}

Finally do some more checks

In [33]:
print rc in pmc # check if the RandomChoiceMover was called in pmc
print rc in pm # check if the pathmover has a RandomChoiceMover at that position in the tree

True
True


> Note, that certain trees of movers can be arranged differently in a tree, but result in the same possible steps. Like `Sequential(Forward, Sequential([Forward, Forward]))` and `Sequential([Forward, Forward, Forward])`. We can regard this as being _associative_ (placing arbitrary brackets) but we will not check for these types of equality. We assume that you arrange steps in a certain way for a reason and that your grouping of sequenced will reflect a certain logical idea. On the other hand are your locators / keys dependend on that choice!

What else can we learn or ask for?

We can check if a particular mover was run. Let's pick the very last MinusExtensionDirectionChoser. To check for it we can either just check for the instance in a pmc

In [34]:
loc_medc = pm.locate('MinusChooser')
medc = pm[loc_medc]
print medc

MinusChooser(10f3a71d0)


In [35]:
%%time
for pc in step_changes:
    if medc in pc:
        print repr(pc)
        print pc.unique
        print

PathSimulator([24])
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- MinusChooser(10f3a71d0)
         +- Minus(1060f5a10)
             +- EnsembleFilter(10f604b90)
                 +- ConditionalSequential(10f604b50)
                     +- MinusSubtrajectoryChooser(10f3b3310)
                     |   +- FinalSubtrajectorySelect(10f3b3c90)
                     +- InterfaceSetChooser(10f3a7110)
                     +- MinusExtensionDirectionChooser(10f604290)
                         +- ForwardExtend(10f3a73d0)

PathSimulator([284])
PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- MinusChooser(10f3a71d0)
         +- Minus(1060f5a10)
             +- EnsembleFilter(10f604b90)
                 +- ConditionalSequential(10f604b50)
                     +- MinusSubtrajectoryChooser(10f3b3310)
                     |   +- FirstSubtrajectorySelect(10f3a7410)
                     +- InterfaceSetChooser(10f3a7110)
                     +- MinusExtensionDirectionChooser(10f604

the expression `locator in pmc` checks for the overall appearance of a specific mover somewhere dependent of the location. While using `mover_instance in pmc` checks for the appearance of that mover instance independent of the location. If a particular mover_instance appears only once then both methods are equal.

In [36]:
print pmc
print loc_medc in pm
print medc in pm
print loc_medc in pmc
print medc in pmc

PathSimulatorStep : PathSampling : Step # 3 with 0 samples
 +- RandomChoice :
     +- RandomChoice :
         +- SampleMove : ReplicaExchangeMover : False :[]
True
True
False
False


In this case the _MinusChooser_ was not called in this particular pmc.

### Remember the following things about moves and changes

1. movers and changes represent a tree like structure of movers.
2. a change is a concrete realization of moves (and hence movers) while a mover itself has a certain degree of choice. 
3. Some nodes in a mover can have different outcomes, like a RandomChoiceMover which will pick one out of several choices. The change it generates will have only the chosen one, while the RandomChoiceMover itself has all possibilities.
4. A concrete tree like structure (without choice) of movers can be represented by a tree of tuples. Reading the tuple-tree from left to right the opening bracket '(' means go down a level (a new child) while the closing bracket means trael up a level ')' go to the parent.
5. The smallest unique tupletree for a pmc can be accessed using `.unique`

In [37]:
first_minus_change = filter(lambda x : p.MinusMover in x, step_changes)[0]

will be interpreted as

In [38]:
print first_minus_change.unique

PathSimulator(10f60df50)
 +- RandomChoice(10f3bb990)
     +- MinusChooser(10f3a71d0)
         +- Minus(1060f5a10)
             +- EnsembleFilter(10f604b90)
                 +- ConditionalSequential(10f604b50)
                     +- MinusSubtrajectoryChooser(10f3b3310)
                     |   +- FinalSubtrajectorySelect(10f3b3c90)
                     +- InterfaceSetChooser(10f3a7110)
                     +- MinusExtensionDirectionChooser(10f604290)
                         +- ForwardExtend(10f3a73d0)


In [39]:
pm.map_tree(lambda x : len(x.name))

(13,
    (12,
        (15,
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (28, (12), (13)),
            (14, (12), (13))),
        (12, (15), (15), (15), (15), (15), (15)),
        (15, (12), (12), (12), (12), (12), (12), (12), (12), (12), (12)),
        (12,
            (5, (14, (21, (25, (24), (24)), (19, (15)), (30, (13), (14))))),
            (5, (14, (21, (25, (24), (24)), (19, (15)), (30, (13), (14))))),
            (5, (14, (21, (25, (24), (24)), (19, (15)), (30, (13), (14)))))),
        (19, (15), (15), (15))))

### Some speed tests

In [40]:
%%timeit
pm.enum

The slowest run took 15.01 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 524 ns per loop


In [41]:
%%timeit
pmc in pm

10000 loops, best of 3: 46.6 µs per loop


In [42]:
%%timeit
[p in pmc for p in pm.enum]

100 loops, best of 3: 2.03 ms per loop


In [43]:
%%timeit
pm in pm

The slowest run took 5.05 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.74 µs per loop


In [44]:
%%timeit
pmc.unique in pm

10000 loops, best of 3: 45.1 µs per loop
